# Libraries and Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Unif and Detail Datasets

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence

# Define paths to folders
IMAGE_UNIFORM_DIR = "/content/drive/MyDrive/OrgUnif"          # Folder for uniform images
MASK_UNIFORM_DIR = "/content/drive/MyDrive/ConvUnif"      # Folder for uniform masks
IMAGE_DETAILED_DIR = "/content/drive/MyDrive/OrgDetail"        # Folder for detailed images
MASK_DETAILED_DIR = "/content/drive/MyDrive/WaterDetail"    # Folder for detailed masks

IMG_HEIGHT = 512  # Set your desired image height
IMG_WIDTH = 512   # Set your desired image width

# Function to load images and masks
def load_images_and_masks(image_dir, mask_dir):
    image_filenames = sorted(os.listdir(image_dir))
    mask_filenames = sorted(os.listdir(mask_dir))

    images = []
    masks = []

    for img_name, mask_name in zip(image_filenames, mask_filenames):
        # Load image and mask
        img_path = os.path.join(image_dir, img_name)
        mask_path = os.path.join(mask_dir, mask_name)

        image = cv2.imread(img_path, cv2.IMREAD_COLOR)  # Read as RGB
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Masks are grayscale

        # Resize image and mask to fixed dimensions
        image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
        mask = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT))

        # Normalize the image (optional, for better training performance)
        image = image / 255.0  # Normalize to range [0, 1]

        # Binarize mask (ensure masks have 0s and 1s)
        mask = np.where(mask > 0, 1, 0).astype(np.float32)

        images.append(image)
        masks.append(mask)

    # Convert to numpy arrays
    images = np.array(images, dtype=np.float32)
    masks = np.array(masks, dtype=np.float32)

    # Add channel dimension to masks (required for U-Net)
    masks = np.expand_dims(masks, axis=-1)

    return images, masks

# Load uniform dataset
uniform_images, uniform_masks = load_images_and_masks(IMAGE_UNIFORM_DIR, MASK_UNIFORM_DIR)
print(f"Loaded {len(uniform_images)} uniform images and masks.")

# Load detailed dataset
detailed_images, detailed_masks = load_images_and_masks(IMAGE_DETAILED_DIR, MASK_DETAILED_DIR)
print(f"Loaded {len(detailed_images)} detailed images and masks.")

# Combine the datasets if needed
all_images = np.concatenate([uniform_images, detailed_images], axis=0)
all_masks = np.concatenate([uniform_masks, detailed_masks], axis=0)

print(f"Total images: {len(all_images)}, Total masks: {len(all_masks)}")


Loaded 291 uniform images and masks.
Loaded 216 detailed images and masks.
Total images: 507, Total masks: 507


In [ ]:
IMG_CHANNELS = 3  # RGB images

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(all_images, all_masks, test_size=0.2, random_state=42)

print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Validation set: {X_val.shape}, {y_val.shape}")

Training set: (405, 512, 512, 3), (405, 512, 512, 1)
Validation set: (102, 512, 512, 3), (102, 512, 512, 1)


In [ ]:
def unet_model(input_size=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)):
    inputs = layers.Input(input_size)

    # Encoder (downsampling)
    c1 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c3)

    # Decoder (upsampling)
    u1 = layers.UpSampling2D((2, 2))(c3)
    u1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u1)
    u1 = layers.Concatenate()([u1, c2])
    c4 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u1)

    u2 = layers.UpSampling2D((2, 2))(c4)
    u2 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(u2)
    u2 = layers.Concatenate()([u2, c1])
    c5 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(u2)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c5)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

# Build and compile the model
model = unet_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 512, 512, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 512, 512, 16)   │            448 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 512, 512, 16)   │          2,320 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 256, 256, 16)   │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 256, 256, 32)   │          4,640 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 256, 256, 32)   │          9,248 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 128, 128, 32)   │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 128, 128, 64)   │         18,496 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 128, 128, 64)   │         36,928 │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ up_sampling2d             │ (None, 256, 256, 64)   │              0 │ conv2d_5[0][0]         │
│ (UpSampling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 256, 256, 32)   │         18,464 │ up_sampling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 256, 256, 64)   │              0 │ conv2d_6[0][0],        │
│                           │                        │                │ conv2d_3[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 256, 256, 32)   │         18,464 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ up_sampling2d_1           │ (None, 512, 512, 32)   │              0 │ conv2d_7[0][0]         │
│ (UpSampling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 512, 512, 16)   │          4,624 │ up_sampling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 512, 512, 32)   │              0 │ conv2d_8[0][0],        │
│ (Concatenate)             │                        │                │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)    

 Total params: 118,273 (462.00 KB)

 Trainable params: 118,273 (462.00 KB)

 Non-trainable params: 0 (0.00 B)

# Save Model

In [ ]:
# Save model
model.save('/content/drive/MyDrive/unet_rbc_segmentation.h5')

# Load model
model = tf.keras.models.load_model('/content/drive/MyDrive/unet_rbc_segmentation.h5')

In [ ]:
# Train the U-Net model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=4
)

Epoch 1/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 1089s 11s/step - accuracy: 0.6361 - loss: 0.6596 - val_accuracy: 0.6148 - val_loss: 0.6684
Epoch 2/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 1082s 11s/step - accuracy: 0.6357 - loss: 0.6598 - val_accuracy: 0.6148 - val_loss: 0.6790
Epoch 3/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 1100s 11s/step - accuracy: 0.6482 - loss: 0.6544 - val_accuracy: 0.6148 - val_loss: 0.6679
Epoch 4/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 1099s 11s/step - accuracy: 0.6535 - loss: 0.6502 - val_accuracy: 0.6148 - val_loss: 0.6677
Epoch 5/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 1110s 11s/step - accuracy: 0.6525 - loss: 0.6478 - val_accuracy: 0.6148 - val_loss: 0.6673
Epoch 6/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 1118s 11s/step - accuracy: 0.6442 - loss: 0.6528 - val_accuracy: 0.6148 - val_loss: 0.6674
Epoch 7/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 1138s 11s/step - accuracy: 0.6363 - loss: 0.6561 - val_accuracy: 0.6148 - val_loss: 0.6672
Epoch 8/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 1140s 11s/step - accuracy: 0.6404 - loss: 0

In [ ]:
# Evaluate the model on validation data
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {accuracy*100:.2f}%")

# Predict on a few images
preds = model.predict(X_val[:15])

# Plot original images, ground truth masks, and predicted masks
for i in range(15):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.title("Original Image")
    plt.imshow(X_val[i])


    plt.subplot(1, 3, 2)
    plt.title("Ground Truth Mask")
    plt.imshow(y_val[i].squeeze(), cmap='gray')

    plt.subplot(1, 3, 3)
    plt.title("Predicted Mask")
    plt.imshow(preds[i].squeeze(), cmap='gray')

    plt.show()